In [3]:
import pandas as pd

# Load datasets
sentiment_df = pd.read_csv("fear_greed_index.csv")
trader_df = pd.read_csv("historical_data.csv")

# Convert timestamps to date format
sentiment_df['date'] = pd.to_datetime(sentiment_df['date']).dt.date
trader_df['trade_date'] = pd.to_datetime(trader_df['Timestamp'], unit='ms').dt.date

# Merge on date
merged_df = pd.merge(trader_df, sentiment_df[['date', 'classification']],
                     left_on='trade_date', right_on='date', how='left')

# Remove trades with no sentiment data
merged_df = merged_df.dropna(subset=['classification'])

# ---- Analysis ----

# 1. PnL grouped by sentiment
pnl_by_sentiment = merged_df.groupby('classification')['Closed PnL'].agg(['mean', 'sum', 'count']).reset_index()

# 2. Avg trade size in USD by sentiment
avg_size_by_sentiment = merged_df.groupby('classification')['Size USD'].mean().reset_index(name='avg_trade_size_usd')

# 3. Trade direction distribution by sentiment
direction_dist = merged_df.groupby(['classification', 'Direction']).size().unstack(fill_value=0)

# Display results
print("PnL by Sentiment:\n", pnl_by_sentiment)
print("\nAverage Trade Size (USD) by Sentiment:\n", avg_size_by_sentiment)
print("\nTrade Direction Distribution by Sentiment:\n", direction_dist)


PnL by Sentiment:
   classification        mean           sum  count
0  Extreme Greed   -9.681505 -7.919471e+03    818
1           Fear  224.481231  2.856973e+06  12727
2          Greed  -56.700945 -8.170606e+04   1441
3        Neutral   76.747074  1.195719e+05   1558

Average Trade Size (USD) by Sentiment:
   classification  avg_trade_size_usd
0  Extreme Greed         5672.919829
1           Fear        18420.351405
2          Greed         7177.487273
3        Neutral         8842.974705

Trade Direction Distribution by Sentiment:
 Direction       Auto-Deleveraging   Buy  Close Long  Close Short  \
classification                                                     
Extreme Greed                   0    88         179           84   
Fear                            0  1158        2471         2422   
Greed                           0   306          89          154   
Neutral                         8     0         586          139   

Direction       Long > Short  Open Long  Open Short